# 2.1 LP Problems

Here is a compendium of ways to run a LP problem in Julia using JuMP and GLPK

Below will be the LP problem that will be used for this example:

$$ \max x_1 + 2x_2 + 5x_3 $$

subject to
$$ -x_1 + x_2 + 3x_3 \leq 5$$
$$ x_1 + 3x_2 - 7x_3 \leq 10$$
$$ 0 \leq x_1 \leq 10 $$
$$ x_2 \geq 0$$
$$ x_3 \geq 0$$

In [63]:
using JuMP, GLPK

In [64]:
# Preparing an optimization model
m = Model(GLPK.Optimizer);

In [65]:
#Declaring Variables 
@variable(m, 0<= x1 <=10);
@variable(m, x2 >= 0);
@variable(m, x3 >= 0);

In [66]:
#Setting objective function
@objective(m, Max, x1 + 2x2 + 5x3);

In [67]:
#adding constraints
@constraint(m, constraint1, -x1 +  x2 + 3x3 <= -5);
@constraint(m, constraint2,  x1 + 3x2 - 7x3 <= 10);

In [68]:
# Printing the prepared optimization model
print(m)

Max x1 + 2 x2 + 5 x3
Subject to
 constraint1 : -x1 + x2 + 3 x3 <= -5.0
 constraint2 : x1 + 3 x2 - 7 x3 <= 10.0
 x1 >= 0.0
 x2 >= 0.0
 x3 >= 0.0
 x1 <= 10.0


In [69]:
# Solving the optimization problem
JuMP.optimize!(m)

In [70]:
# Printing the optimal solutions obtained

println("Objective value: ", JuMP.objective_value(m))


println("Optimal Solutions:")
println("x1 = ", JuMP.value(x1))
println("x2 = ", JuMP.value(x2))
println("x3 = ", JuMP.value(x3))

Objective value: 19.0625
Optimal Solutions:
x1 = 10.0
x2 = 2.1875
x3 = 0.9375


<b>Shadow prices</b>: Calculating the effect on the objective value if the RHS of the constraint equation is increased by one. 

In [71]:
# Printing the optimal dual variables
println("Dual Variables:")
println("dual1 = ", JuMP.shadow_price(constraint1))
println("dual2 = ", JuMP.shadow_price(constraint2))

Dual Variables:
dual1 = 1.8125
dual2 = 0.06249999999999998


# 2.2 Alternative Ways of Writing LP

In [72]:
using JuMP, GLPK

In [73]:
# Preparing an optimization model
m = Model(GLPK.Optimizer);

In [74]:
# Setting Variable x
@variable(m, x[1:3] >= 0);

In [75]:
# Setting Variable C and Objective function
C = [1,2,5]

@objective(m, Max, sum(C[i]*x[i] for i in 1:3));

In [76]:
# Constraints definition in the form of a Matrix
A = [-1  1  3;
      1  3 -7];
b = [-5; 10];
@constraint(m, constraint1, sum( A[1,i]*x[i] for i in 1:3) <= b[1] );
@constraint(m, constraint2, sum( A[2,i]*x[i] for i in 1:3) <= b[2] );

Below are alternative ways to defining constraints. Uncomment to see how it works. To unregister constraints, use 'unregister(model, :constraint#)'

In [77]:
#-------------Alternative ways of coding constraints----------------------------------


#-------------Alternative 1-----------------------------------------------------------
# constraint = Dict()
# for j in 1:2
#   constraint[j] = @constraint(m, sum(A[j,i]*x[i] for i in 1:3) <= b[j])
# end


#-------------Alternative 2-----------------------------------------------------------
# @constraint(m, constraint[j in 1:2], sum(A[j,i]*x[i] for i in 1:3) <= b[j])

In [78]:
#adding bounds for the value of x[1]
@constraint(m, bound, x[1] <=10);

In [79]:
JuMP.optimize!(m)

In [80]:
#Printing the optimal values

println("Objective value: ", JuMP.objective_value(m))


println("Optimal Solutions:")
for i in 1:3
    println("x[$i] = ", JuMP.value(x[i]))
end

Objective value: 19.0625
Optimal Solutions:
x[1] = 10.0
x[2] = 2.1875
x[3] = 0.9375


In [81]:
# placing shadow prices of individual constraints in a list for printing
constraint = [JuMP.shadow_price(constraint1), JuMP.shadow_price(constraint2)]

#Printing the dual variables
println("Dual Variables:")
for j in 1:2
  println("dual[$j] = ", constraint[j])
end

Dual Variables:
dual[1] = 1.8125
dual[2] = 0.06250000000000003


# 2.3 Yet Another Way of Writing LP Problems

In section 2.2, to change the data and solve another probelm with the same structure, individual lists would need to be updates. This could be rather tedious. Here is another way to update values.

In [82]:
using JuMP, GLPK

In [83]:
# Preparing an optimization model
m = Model(GLPK.Optimizer);

In [84]:
# adding indices to track constraints
index_x = 1:3
index_constraints = 1:2

A = [-1  1  3;
      1  3 -7];
b = [-5; 10];
c = [ 1; 2; 5];

In [85]:
@variable(m, x[index_x] >= 0);
@objective(m, Max, sum( c[i]*x[i] for i in index_x) );

#---------New way to define constraints--------------------
@constraint(m, constraint[j in index_constraints], 
                            sum( A[j,i]*x[i] for i in index_x ) <= b[j] );

# 2.4 Mixed Integer Linear Programming (MILP) Problems

For MILP problems, GUROBI and CPLEX optimization solvers will be used to solve MILP problems.

Some changes will be made based on the original LP problem as an example for MILP. The following problem will be solved using Julia:

$$ \max x_1 + 2x_2 + 5x_3$$

subject to

$$ -x_1 + x_2 + 3x_3 <= 5$$
$$ x_1 + 3x_2 - 7x_3 <= 10$$
$$ 0 <= x_1 <= 10 $$
$$ x_2 >= 0 \ Integer$$
$$ x_3 \in \{0,1\} .$$

In [86]:
using JuMP, Gurobi

In [87]:
# Preparing an optimization model
m = Model(Gurobi.Optimizer);

Set parameter Username
Academic license - for non-commercial use only - expires 2023-11-10


In [88]:
# defining integer and binary variables
@variable(m, 0<= x1 <=10);
@variable(m, x2 >=0, Int);
@variable(m, x3, Bin);

In [89]:
# Setting the objective
@objective(m, Max, x1 + 2x2 + 5x3);

# Adding constraints
@constraint(m, constraint1, -x1 +  x2 + 3x3 <= -5);
@constraint(m, constraint2,  x1 + 3x2 - 7x3 <= 10);

In [90]:
# Printing the prepared optimization model
print(m)

Max x1 + 2 x2 + 5 x3
Subject to
 constraint1 : -x1 + x2 + 3 x3 <= -5.0
 constraint2 : x1 + 3 x2 - 7 x3 <= 10.0
 x1 >= 0.0
 x2 >= 0.0
 x1 <= 10.0
 x2 integer
 x3 binary


In [91]:
# Solving the optimization problem
JuMP.optimize!(m)

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 2 rows, 3 columns and 6 nonzeros
Model fingerprint: 0x001f10fa
Variable types: 1 continuous, 2 integer (1 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+00]
  Objective range  [1e+00, 5e+00]
  Bounds range     [1e+01, 1e+01]
  RHS range        [5e+00, 1e+01]
Found heuristic solution: objective 19.0000000
Presolve removed 2 rows and 3 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 16 available processors)

Solution count 1: 19 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.900000000000e+01, best bound 1.900000000000e+01, gap 0.0000%

User-callback calls 240, time in user-callback 0.00 sec


In [92]:
# Printing the optimal solutions obtained

println("Objective value: ", JuMP.objective_value(m))

println("Optimal Solutions:")
println("x1 = ", JuMP.value(x1))
println("x2 = ", JuMP.value(x2))
println("x3 = ", JuMP.value(x3))

Objective value: 19.0
Optimal Solutions:
x1 = 10.0
x2 = 2.0
x3 = 1.0
